# EECS 504 Problem Set 1

__Please provide the following information__
(e.g. Jason Corso, jjcorso):

[Your first name] [Your last name], [Your UMich uniqname]

__Important__: after you download the .ipynb file, please name it as EECS504_PS1_\<your_uniquename\>_\<your_umid\>.ipynb before you submit it to canvas. Example: EECS504_PS1_adam_01101100.ipynb.



# Introduction

We'll provide you with starter code, like this, in a Jupyter notebook for most problem sets. Please fill in the code to complete the assignment, and submit your notebook to Canvas as a .ipynb file. You can, of course, initially write your code offline in an editor like Emacs or Vim -- we'd just like the final output to be in a notebook format to make grading more consistent.

Please note that *we won't run your code*. The notebook you submit should already contain all of the results we ask for. In other words, outputs should be computed *before you submit*.  Also, please do not include long, unnecessary outputs (a few print statements and visualizations are fine, but pages of debugging messages make grading difficult).

If you'd like to preview what your notebook will look like when we grade it, follow the directions [here](https://stackoverflow.com/questions/15998491/how-to-convert-ipython-notebooks-to-pdf-and-html) for converting the .ipynb notebook into an HTML file. We'll be running a command similar to: 
```
ipython nbconvert --to html EECS_504_PS1.ipynb  
```

# Starting

Run the following code to import the modules you'll need. After your finish the assignment, remember to run all cells and save the note book to your local machine as a .ipynb file for Canvas submission.

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import IPython

In [ ]:
%%capture
! wget -O img1.jpg "https://drive.google.com/uc?export=download&id=1dDdk7IrM5oO2gA57_bapqdIgk7CmJV03"
! wget -O img2.jpg "https://drive.google.com/uc?export=download&id=1V_lui-asfT3Xly4O_Kuez8IDYEZkct4i"
! wget -O orion.jpg "https://drive.google.com/uc?export=download&id=1L1deUWfJbdtss4liHii8Luhnl2RG_ITl"
! wget -O point_tracker.html "https://drive.google.com/uc?export=download&id=1I9Xw8u0EdAEsP4GGuNwh267zL5lwxQw_"


#Problem 1.3. Calculating NFL's pitch homography


# Visualize Input Images

In [ ]:
img1 = cv2.imread('img1.jpg')
img2 = cv2.imread('img2.jpg')

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

img1 = cv2.resize(img1,(575, 313))
img2 = cv2.resize(img2,(575, 313))


def plot_imgs(img1, img2):
  fig, ax = plt.subplots(1, 2, figsize=(150, 200))
  for a in ax:
    a.set_axis_off()
  ax[0].imshow(img1)
  ax[1].imshow(img2)

plot_imgs(img1, img2)

# Record point correspondences 

To obtain the homography between two images you need to record multiple point correspondences. You can gather as many correspondences as you want as long as there are more than the minimum you calculated in Problem 1.2.

We have provided you with a point tracker tool that shows you the x and y coordinates of your cursor on top of any of the images. It also records your last clicks, so it should pretty easy for you to alternate clicking between the two images. You will use the points recorded to build your A and y vector later.

In [ ]:
IPython.display.HTML('./point_tracker.html')

#Use the correspondences to build A and y

Use the recorded point correspondences to build your A matrix and y vector based on the structure that you obtained in problem 1.1.




In [ ]:
# ===============================================
# TODO


# A = np.array([[]]) 
# y = np.array([[]]) 


# ===============================================


# Use A and y to find your homgraphy

Now that you have your A matrix and y vector you can find your homography by applying the least squares formula to solve for x. Then build your 3x3 homography matrix H from the values of the vector x.

In [ ]:
# ===============================================
# TODO

# x =  
# H =

# ===============================================

It is time to check if you got the right homography!

Run the code below. The image on the left has the yellow line at the 33rd yard. If your homography is correct the yellow line in the second image should also be at the 33rd yard. 

In [ ]:

line_1 = ((int(1210/3), int(126/3)), (int(1701/3), int(939/3)))

#First point of second line
pt1 = H@np.array([[line_1[0][0]],[line_1[0][1]], [1]])
pt1 /= pt1[2,0]

#Second point of second line
pt2 = H@np.array([[line_1[1][0]],[line_1[1][1]], [1]])
pt2 /= pt2[2,0]

line_2 = ((pt1[0], pt1[1]), (pt2[0], pt2[1]))


img1 = cv2.line(img1, (line_1[0][0],line_1[0][1]), (line_1[1][0], line_1[1][1]), (255, 255, 0) , 4)
img2 = cv2.line(img2, (line_2[0][0],line_2[0][1]), (line_2[1][0],line_2[1][1]), (255, 255, 0) , 4)

plot_imgs(img1, img2)

#Problem 4. Bayer Demosaicking

In this problem you will implement the function demosaic to recover an image from its grayscaled mosaicked version. We wrote the mosaicking function bayer_filter for you. We encourage you to try to understand how it works.


#Visualize original image

In [ ]:
orion_img = cv2.imread('orion.jpg')
orion_img = cv2.cvtColor(orion_img, cv2.COLOR_BGR2RGB)
plt.figure()
plt.axis('off')
plt.imshow(orion_img)

#Visualize image after bayer filter is applied (Colored and greyscale)

In [ ]:
def bayer_filter(img):
  # function bayer_filter filters an input color image I 
  # by the Bayer pattern as below, and outputs a filtered color 
  # image I and a gray scaled image I_gray in which the 
  # value of every pixel is the intensity of corresponding
  # color.
  # b | g
  # -----
  # g | r

  
  rows, cols, _ = img.shape

  colored = img.copy()

  mask = np.zeros((2,2,3))
  mask[0,0,:] = [0,0,1] #blue
  mask[0,1,:] = [0,1,0] #green
  mask[1,0,:] = [0,1,0] #green
  mask[1,1,:] = [1,0,0] #red

  for row in range(rows):
    mask_row = row%2

    for col in range(cols):
      mask_col = col%2
      colored[row, col, :] = colored[row, col, :] * mask[mask_row, mask_col, :]

  gray = colored[:, :, 0] + colored[:, :, 1] + colored[:, :, 2]

  return colored, gray
  

   
filtered_img, gray_img = bayer_filter(orion_img)

fig, ax = plt.subplots(1, 2, figsize=(150, 200))
for a in ax:
  a.set_axis_off()
ax[0].imshow(filtered_img)
ax[1].imshow(gray_img, cmap="gray")




#Demosaicking

Now you are going to build your demosaicking function. We recommend that you write the helper functions in the block below first because it will make things easier for you. These functions are meant to compute the average of the sorrounding pixels in all possible scenarios.

In [ ]:
# ===============================================

# def four_neighbors_average(gray_img, row, col):
  # Computes the average of the four neighborgh pixels (left, right, top and bottom).
  # REMEMBER that for the pixels on the border, we only consider the neighboring pixels within the image domain.

  # TODO (Optional)

# def four_corners_average(gray_img, row, col):
  # Computes the average of the four corner pixels (top-left, top-right, bottom-left, bottom-right).
  # REMEMBER that for the pixels on the border, we only consider the neighboring pixels within the image domain.

  # TODO (Optional)

# def left_right_average(gray_img, row, col):
  # Computes the average of the left and right pixels.
  # REMEMBER that for the pixels on the border, we only consider the neighboring pixels within the image domain.

  # TODO (Optional)

# def top_bottom_average(gray_img, row, col):
  # Computes the average of the top and bottom pixels.
  # REMEMBER that for the pixels on the border, we only consider the neighboring pixels within the image domain.

  #TODO (optional)

# ===============================================

In [ ]:
# ===============================================

def demosaic(gray_img):
  # recovers the original color image (M*N*3) from the Bayer encoded image I_gray (M*N)
  pass
  #TODO

# ===============================================


#Visualize reconstructed image

Time to see if your algorithm works. Run the code below to check if the reconstructed image matches the original one.

In [ ]:
recon_img = demosaic(gray_img)
plt.figure()
plt.axis('off')
plt.imshow(recon_img/255.)

#Optional
 Upload your own picture. Use the bayer_filter function that we wrote to make a mosaic of the image you uploaded. Then apply the demosaic function to see the results. There will be no grading for this question.

In [ ]:
# ===============================================
#TODO: apply bayer filter to your own uploaded image,
# then apply demosaic function and show the results.


# ===============================================